In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/MusiCAN

/content/drive/MyDrive/Github/MusiCAN


In [ ]:
!git pull

In [5]:
# colab version: run this code..

!pip install muspy

"""
non-colab version: install
1. muspy via pip install in environment folder (e.g. /Users/kai/anaconda3/opt/envs/MusiCAN/bin)
2. maybe fluidsynth via conda install -c conda-forge fluidsynth
"""

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 119 kB 4.2 MB/s 
     |████████████████████████████████| 5.6 MB 34.9 MB/s 
     |████████████████████████████████| 51 kB 6.7 MB/s 
     |████████████████████████████████| 22.4 MB 1.3 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=e0b3ea5549a440a2e47dadec7f0b3ee90c893edbd16cbab14342e5b7b9eaecee
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
  Attempting uninstall: music21
    Found existing installation: music21 5.5.0
    Uninstalling music21-5.5.0:
      Successfully uninstalled music21-5.5.0


'\nnon-colab version: install\n1. muspy via pip install in environment folder (e.g. /Users/kai/anaconda3/opt/envs/MusiCAN/bin)\n2. maybe fluidsynth via conda install -c conda-forge fluidsynth\n'

In [6]:
from IPython.display import Audio, display

import os
import os.path
import json
import random
import datetime
from tqdm import tqdm # valuebar for iterations

import numpy as np
import matplotlib.pyplot as plt
import torch

import muspy
import pypianoroll
from pypianoroll import Multitrack, Track

In [7]:
# run this cell if you didn't do it already 
muspy.download_musescore_soundfont() 

Start downloading MuseScore General soundfont.
MuseScore General soundfont has successfully been downloaded to : /root/.muspy/musescore-general.


In [8]:
# run this cell if you didn't do it already 
muspy.download_bravura_font() 

Start downloading Bravura font.
Bravura font has successfully been downloaded to : /root/.muspy/musescore-general.


In [10]:
%cd /content/drive/MyDrive/Github/MusiCAN/data_preparation

/content/drive/MyDrive/Github/MusiCAN/data_preparation


In [11]:
# create labels & id_list

genre_list = ['Rap', 'Latin', 'International', 'Electronic', 
              'Country', 'Folk', 'Blues', 'Reggae', 'Jazz',
              'Vocal', 'New-Age', 'RnB', 'Pop_Rock'] # genre <-> numeric label = index

id_list = [] # id = MillionSongsDataset ID
track_label_list = []
for path in os.listdir("unprepared_data/id_lists_amg"):
    filepath = "unprepared_data/id_lists_amg/" + path
    
    with open(filepath) as f:
        ids = [line.rstrip() for line in f]
        number_of_ids = len(ids)
        id_list.extend(ids)
    
    genre_no = genre_list.index(path[8:-4])
    track_label_list.extend([genre_no] * number_of_ids)  

In [13]:
print(track_label_list)

[6, 6, 6, 4, 4, 4, 3, 3, 3, 5, 5, 5, 2, 2, 2, 8, 8, 8, 1, 1, 1, 10, 10, 10, 12, 12, 12, 0, 0, 0, 7, 7, 7, 11, 11, 11, 9, 9, 9]


In [14]:
# make sure no multiple genre label 
n = 0
id_array = np.array(id_list)
for id_1 in id_list:
    n_ids = np.sum(id_array == id_1)
    n += (n_ids - 1)
n == 0

158


In [ ]:
# helper function

def msd_id_to_dirs(msd_id):
    """Given an MillionSongsDataset ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return(msd_id[2] + "/" + msd_id[3] + "/" + msd_id[4] + "/" + msd_id)

In [ ]:
# helper function

def pianoroll_plot(multitrack):
    """Given muspy.multitrack object, plot part of pianoroll"""
    multitrack_copy = multitrack.copy()
    multitrack_copy.trim(end=12 * 96)
    axs = multitrack_copy.plot()
    plt.gcf().set_size_inches((16, 8))
    for ax in axs:
        for x in range(96, 12 * 96, 96):     
            ax.axvline(x - 0.5, color='k', linestyle='-', linewidth=1)
    plt.show()

In [ ]:
# prepare data

# set values
n_pitches = 7*12  # number of pitches
lowest_pitch = 2*12  # MIDI note number of the lowest pitch
beat_resolution = 12 # temporal resolution of a beat (in timestep), 24 in data, 12 for MusiGAN
sample_size = 4*4 # number of beats per instance created by track-cropping, 4 bars for MusiGAN
min_n_notes = 8 # minimal number of notes per instance

dataset_root = "unprepared_data/lpd_5/lpd_5_cleansed/"

# iterate over all the songs in the ID list
data_list = []
data_label_list = []

for i, msd_id in enumerate(tqdm(id_list)):
    
    # load multitrack as a pypianoroll.Multitrack instance
    song_dir = dataset_root + msd_id_to_dirs(msd_id)
    filename = os.listdir(song_dir)[0]
    multitrack = pypianoroll.load(song_dir + "/" + filename)
    
    # binarize pianorolls
    multitrack.binarize()
    
    # remove trailing silence: multitrack.trim()
    
    # downsample pianorolls (shape: time x pitches)
    multitrack.set_resolution(beat_resolution)

    # array conversion (shape: tracks x time x pitches) & extract piano track (= 2nd track) 
    pianoroll = multitrack.stack()[1,:,:]
    
    # fix pitch range
    pianoroll = pianoroll[:, lowest_pitch : lowest_pitch + n_pitches] # (shape: time x pitches))
    
    # crop pianoroll into smaller training samples
    n_timesteps = sample_size * beat_resolution # time steps per instance
    pianoroll = pianoroll[ : pianoroll.shape[0] - (pianoroll.shape[0] % n_timesteps), :] # make sure: number of total timesteps of track % n_timesteps == 0, else skip last bar
    pianoroll = pianoroll.reshape((-1, n_timesteps, n_pitches))
    
    # append instances with number of notes >= minimal number of notes (min_n_notes)
    good_instances_mask = (pianoroll.sum(axis = 1).sum(axis = 1) >= min_n_notes)
    data_list.append(pianoroll[good_instances_mask])
    
    # append labels
    data_label_list.extend([track_label_list[i]] * np.sum(good_instances_mask))
    
data_array = np.concatenate(data_list, axis = 0) # (shape: n_instances x n_timesteps x n_pitches)
label_array = np.array(data_label_list)
print(f"Successfully collect {len(data_array)} samples from {len(id_list)} songs")
print(f"Data shape : {data_array.shape}, {label_array.shape}")


# create unique file directory to save data
timestamp = datetime.datetime.now()
file_directory = f"./prepared_data/lpd5_{timestamp}"
os.makedirs(file_directory)
os.makedirs(file_directory + "/audio_examples") # for later..

# save preparation parameters as json file
prep_pars_dict = {"n_pitches": n_pitches,
                 "lowest_pitch": lowest_pitch,
                 "beat_resolution": beat_resolution, 
                  "beats_per_instance": sample_size,
                  "minimal_number_of_notes_per_instance": min_n_notes}
with open(file_directory + "/preparation_params.json", "w") as file:
    json.dump(prep_pars_dict, file, indent = 6)

# save data as compressed npz files
np.savez_compressed(file_directory + "/prepared_arrays.npz", data=data_array, labels=label_array)

In [ ]:
# load data
batch_size = 15
file_directory = file_directory # change if wanted

loaded_data = np.load(file_directory + "/prepared_arrays.npz")
loaded_data_array, loaded_label_array = loaded_data["data"], loaded_data["labels"]

# convert to pytorch tensor
data_tensor = torch.as_tensor(loaded_data_array, dtype=torch.float32)
label_tensor = torch.as_tensor(loaded_label_array, dtype=torch.int)

# create pytorch dataset & dataloader
dataset = torch.utils.data.TensorDataset(data_tensor, label_tensor)
lpd5_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
# convert random instances of loaded data to wave (audio) file & display them

n = 15 # number or random examples
rand_idxs = np.random.randint(0, len(loaded_label_array), n)

for i in tqdm(rand_idxs):
    X, y = loaded_data_array[i, :, :], loaded_label_array[i]
    
    genre_of_X = genre_list[y]
    
    X_padded = np.pad(X, ((0, 0), (lowest_pitch, 128 - lowest_pitch - n_pitches))) # complete pitch range
    X_music = muspy.from_pianoroll_representation(X_padded > 0, 
                resolution = beat_resolution, encode_velocity = False) # convert to muspy.music_object

    X_timestamp = datetime.datetime.now()
    muspy.write_audio(path = file_directory + f"/audio_examples/{genre_of_X}_{X_timestamp}.wav", 
                      music = X_music) 
    
    # display audio & show pianoroll
    print(genre_of_X + ":")
    display(Audio(filename = file_directory + f"/audio_examples/{genre_of_X}_{X_timestamp}.wav"))
    muspy.visualization.show_pianoroll(X_music)

In [ ]:
# number of data for each genre?

plt.hist(loaded_label_array)
plt.ylabel("# of instances")
plt.xlabel("genre label")

print(genre_list)